# Extensions

The wrapper class functionalities can be extended using composition or inheritance.

## Composition

The concept of composition adds a component to a composite class. The relationship is such, that the composite class does not know the component class but wise versa. This allows to add additional functionality without changing the original code. Adding components to an existing wrapper class will be called "registering" here.

### Registration of properties
To facilitate and speed up certain tasks and workflows, it might be usefull to have additional properties. Let's assume we want to store a username as an attribute in the HDF5 file. Let's further assume that a user name has a firstname and a surname and both must start with a capitalized letter. To catch possible errerneous input by the user, we would need an additional method, e.g. "set_username" or a property "username" of `H5File`.<br>**Two** possible ways to achieve this: **inheritance or composition**. The quick and recommended way to add a new class property is to use composition by "registering" the property. Like this we don't have to rewrite (inherite at least) the API but only simply write the porperty class.<br>
Let's first write a property class "username". It needs **three methods**: `get`, `set` and `delete`:

In [ ]:
import h5rdmtoolbox as h5tbx
h5tbx.use('cflike')

In [ ]:
from typing import Tuple

# register a property with setter and getter method:
@h5tbx.conventions.register_hdf_attr(h5tbx.wrapper.cflike.H5Dataset, overwrite=True, name='username')
class username:
    """User Name Property. Requires name and ORCID"""

    def set(self, user_name_data: Tuple[str, str]):
        """setter method"""
        if not isinstance(user_name_data, (tuple, list)):
            raise ValueError(f'Expecting a tuple of name and orcid but got {type(user_name_data)}')
        if not len(user_name_data) == 2:
            raise ValueError(f'Expecting two entries (name and orcid) but got {len(user_name_data)}: {user_name_data}')
        self.attrs['user_name'] = user_name_data[0]
        self.attrs['user_orcid'] = user_name_data[1]

    def get(self):
        """getter method"""
        return self.attrs['user_name'], self.attrs['user_orcid']

Some words about above lines:
- `@h5tbx.wrapper.register_special_property(h5tbx.H5File)`: registering the below class (only) to the class `H5File`
- don't use existing property names. An error will be raised anyhow. You may however pass `overwrite=True` in the registration method. Be careful though!
- provide `set` and optionally also `get` and `delete`. The method `get` makes sense though, while `delete` is not really needed most of the times.

Let's check if it worked out:

In [ ]:
with h5tbx.H5File() as h5:
    ds = h5.create_dataset('test', shape=(2,), units='', standard_name='standard_test')    
    ds.username = 'First User', '0000-0001-8729-0482'
    print(ds.username)
    try:
        ds.username = 'Second User'
    except ValueError as e :
        print(f'Could not add user due to: {e}')
    print(ds.username)
    h5.dump()

### Registration of datasets
We can also "register" dataset accessors. In the following example we add "device" as a "property with methods". So "device" seems to be a property which has a method "add". Such an implementation faclitates the interaction with HDF data, too. Note, that this "property-like" accessor is available for all `H5Dataset` objects from know on in this session:

In [ ]:
from h5rdmtoolbox.wrapper.accessory import register_special_dataset
@register_special_dataset('device', h5tbx.H5File.H5Dataset(), overwrite=True)
class DeviceProperty:
    """Device Accessor class"""

    def __init__(self, ds):
        self._ds = ds
        self._device_name = 'NoDeviceName'
        
    def add(self, new_device_name):
        """adds the attribute device_name to the dataset"""
        self._ds.attrs['device_name'] = new_device_name
        
    @property
    def name(self):
        return self._ds.attrs['device_name']

In [ ]:
with h5tbx.H5File() as h5:
    ds = h5.create_dataset('test', shape=(2,), units='', standard_name='standard_test')
    print(type(ds))
    ds.device.add('my device')
    print(ds.device.name)

## Inheritance 

### New Wrapper Class

The main wrapper class around a HDF5 file in this package is `H5File` which uses the wrapper class `H5Group` for `h5py.Group` and `H5Dataset` for wrapping `h5py.Dataset`

In [ ]:
import h5rdmtoolbox as h5tbx
import h5py

In [ ]:
class MyDataset(h5tbx.wrapper.core.H5Dataset):
    def __repr__(self):
        return f'{self.name}\n{self.attrs}'
    
    @property
    def is_2d(self):
        """returns whether dataset is two-dimensional or not"""
        return self.ndim == 2

Next we create a group class with a special method, hat returns all datasets of that group. Also the `create_dataset` method is overwritten. Take care to return `MyDataset` at the end of the method, otherwise dataset class of the parent class is taken.

In [ ]:
class MyGroup(h5tbx.wrapper.core.H5Group):
    
    def get_all_datasets(self):
        """returns all datasets of this group"""
        return [k for k in self if isinstance(self[k], h5py.Dataset)]
    
    def create_group(self, *args, **kwargs):
        return __class__(super().create_group(*args, **kwargs).id)
    
    # def create_dataset(self, *args, **kwargs):
    #     return __class__(super().create_dataset(*args, **kwargs).id)
    
    def create_dataset(self, name, user, *args, **kwargs):
        ds = super().create_dataset(name, *args, **kwargs)
        ds.attrs.modify('user', user)
        return self._h5ds(ds.id)

The main file wrapper inherites from `H5File` ("root" parent was `h5py.File`) and the new group class. Next, we have set the group and dataset class again, since some methods in the file wrapper class will need that information when returning instances of those classes (e.g. dataset or group creation). Finally we define a new method which sets the user name to the root attributes:

In [ ]:
class MyWrapper(h5tbx.wrapper.core.H5File, MyGroup):
    
    def set_user(self, user_name):
        self.attrs.modify('user', user_name)

register the dataset and group class in all classes. This is needed, so all return objects are of the newly defined types

In [ ]:
MyGroup._h5ds = MyDataset
MyGroup._h5grp = MyGroup

MyDataset._h5ds = MyDataset
MyDataset._h5grp = MyGroup

In [ ]:
h5 = MyWrapper()

In [ ]:
type(h5)

In [ ]:
g = h5.create_group('grp', overwrite=True)

In [ ]:
type(g)

In [ ]:
gg = g.create_group('grp', overwrite=True)

In [ ]:
type(gg)

In [ ]:
ds = gg.create_dataset('hello', shape=(2,3), user='Max', overwrite=True)

In [ ]:
ds

In [ ]:
type(ds)

In [ ]:
h5.set_user('new_user')

In [ ]:
h5